In [53]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [54]:
offers_connection = sqlite3.connect('../datasets/offers_dataset.db')
offers_frame = pd.read_sql_query('''SELECT * FROM offers''', offers_connection)
offers_connection.close()

# [!] Le descrizioni all'interno del notebook non sono mostrate nella loro versione completa
#     per accedere ad una specifica descrizione usare il metodo: offers_frame.loc[ID, NomeColonna]
offers_frame

,Name,Description,Location
0,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria"
1,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona
2,Software Developer,We are looking for talented and passionate peo...,55100 Lucca
3,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata"
4,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia"
...,...,...,...
165,Data Scientist - Pharma,Akkodis è un leader globale nel mercato dell'i...,"Provincia di Torino, Piemonte"
166,DATA SCIENTIST | MACHINE LEARNING | BIG DATA,Techyon: Information Technology Recruitment Ex...,"Parma, Emilia-Romagna"
167,PDS AD&GMP Scientist I,Job DescriptionWhen you’re part of Thermo Fish...,20900 Monza
168,Junior Ethical Data Scientist,We are looking for an extremely BOLD & VISIONA...,None


## Data Exploration
Come prima cosa esaminiamo il nostro dataset di offerte, in particolare dovremmo verificare la qualità dei dati:
- Base   
    - Presenza di valori nulli
    - (**Importante**) Presenza di sinonimi geografici
    - Presenza di informazioni utili
- Avanzato
    - Analisi delle Parole Chiave (es. Titoli)
    - Analisi della distribuzione dei dati

**Presenza di Valori Nulli e Duplicati**
Come è possibile vedere di seguito il dataset contiene valori nulli solo per la colonna Location, il che però è normale, in quanto non tutte le Offerte di lavoro sono in presenza, quindi le Location mancanti rappresentano il numero di Offerte di lavoro in Remote.
Questa informazione è importante perchè in seguito ci sarà la necessità di rappresentare queste informazioni in modo omogeneo, in modo tale che un modello il cui scopo è raggruppare le Offerte (clustering, come proposto nell'Analisi del Problema) possa farlo in modo adeguato.

In [55]:
offers_frame.dropna(inplace=True)
offers_frame.drop_duplicates(inplace=True)
offers_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 0 to 169
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         134 non-null    object
 1   Description  134 non-null    object
 2   Location     134 non-null    object
dtypes: object(3)
memory usage: 4.2+ KB


## Presenza di Sinonimi Geografici
Il dataset contiene attualmente molteplici istanze in cui la stessa Location è riportata in modi diversi, come \<Cap>, \<Città> e \<Città>, \<Regione>.


In [56]:
#example_city = "Bologna"

#onsite_offers = offers_frame[offers_frame["Location"].notna()]
#geo_syn = onsite_offers.Location.map(lambda text: example_city in text)
#onsite_offers[geo_syn]

**Analisi dei Titoli**
Di seguito vengono analizzati i Titoli delle Offerte di lavoro con lo scopo di verificare la presenza di termini comuni, in particolare vengono usate delle basilari tecniche di Natural Language Processing come:
- **Tokenization**, utilizzata per dividere frasi o documenti in parole (*title.split()*) 
- **Rimozione della Punteggiatura**, utilizzata per migliorare la qualità dei dati estratti, in modo da non influenzare i risultati 
- **Normalizzazione**, utilizzata per ridurre la complessità del vocabolario e trattare parole con maiuscole e minuscole come equivalenti (*t.lower()*)

In [57]:
from string import punctuation
from collections import Counter

def count_words(document: str, counter: Counter):
    """
    :param document: a string representation of the document to inspect
    :param counter: a Python built-in object used to count occurrences of items
    """
    punct = [p for p in punctuation]
    for t in document.split():
        if not t in punct:
            counter[t.lower()] += 1


titles_counts = Counter()
offers_frame.Name.map(lambda text: count_words(text, titles_counts))

titles_counts

Counter({'developer': 42,
         'software': 35,
         'engineer': 30,
         'network': 19,
         'junior': 14,
         'web': 14,
         'data': 10,
         'senior': 9,
         'intern': 9,
         'sviluppatore': 8,
         '2024': 8,
         'full': 7,
         'scientist': 7,
         'analyst': 6,
         'programmatore': 5,
         'graduate': 5,
         'learning': 5,
         'development': 4,
         'c#': 4,
         'security': 4,
         'frontend': 4,
         'stack': 4,
         'backend': 4,
         'remote': 3,
         'engineering': 3,
         'apprenticeship': 3,
         '(london)': 3,
         'system': 3,
         'summer': 3,
         'internship': 3,
         '–': 3,
         'consultant': 3,
         'it': 3,
         'machine': 3,
         'stage': 2,
         'java': 2,
         'settore': 2,
         'sanità': 2,
         'uk': 2,
         'technology': 2,
         'engineer,': 2,
         'programming': 2,
         'cyber': 2,
  

Analisi delle occorrenze di parole:

In [58]:
# Jacopo plotta qui

## Data Preparation
Per permettere il completo soddisfacimento dei requisiti, alcuni dati mancanti alle offerte dovranno essere aggiunti al dataset, tra cui:
- RequiredSkills: le Competenze richieste da una Offerta
- LocationType: il tipo di Location, se in remoto (Remote) o in sede (OnSite)
- Languages: le lingue naturali necessarie per l'offerta di lavoro

### Skill Extraction
Per procedere con l'estrazione delle Competenze dalle Offerte per prima cosa importiamo il relativo dataset.   

In [59]:
# [!] In alcune descrizioni di offerte c'è HTML5 o CSS3 che non vengono riconosciuti come HTML o CSS

with sqlite3.connect('../datasets/skills_dataset.db') as skills_connection:
    skills_frame = pd.read_sql_query('SELECT * FROM skills', skills_connection)
    skills_frame.set_index('ID', inplace=True)
    
skills_frame

,SKILL,TYPE
ID,,
0,JavaScript,Programming Language
1,HTML,Programming Language
2,Python,Programming Language
3,SQL,Programming Language
4,TypeScript,Programming Language
...,...,...
95,APT,Tool
96,Unity 3D,Tool
97,Pacman,Tool


In [60]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def remove_symbols(description: str, remove_map: dict) -> str:
    """
    """
    for old, new in remove_map.items():
        description = description.replace(old, new)
    return description.lower()

def extract_symbols(text: str, available_symbols: list) -> set:
    """
    """
    out = set()
    for token in set(word_tokenize(text)):
        if token.lower() in available_symbols:
           out.add(token)
    return out

"""
def extract_symbols(description: str, available_symbols: list) -> set:
    s = set()
    for word in description.split():
        if word in available_symbols:
            s.add(word)
    return s 
"""

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'\ndef extract_symbols(description: str, available_symbols: list) -> set:\n    s = set()\n    for word in description.split():\n        if word in available_symbols:\n            s.add(word)\n    return s \n'

In [61]:
# Get Skill list
skills_list = [skill.lower() for skill in skills_frame['SKILL'].tolist()]

# Symbols to remove
punct = [p for p in punctuation]
punct.remove('+')
punct.remove('#')
punct.remove('.')

removal = {p: '' for p in punct}
removal['\n'] = ''
removal['/'] = ' '
removal['('] = ' '
removal[')'] = ' '
removal[','] = ' '

In [65]:
# offers_frame['RequiredSkills'] = ''
with_no_skills = 0

for i, offer_description in enumerate(offers_frame.loc[:, 'Description']):
    desc = remove_symbols(offer_description, removal)
    offer_skills = extract_symbols(desc, skills_list)
    if not offer_skills:
        with_no_skills += 1
        # print(f'{i}: No Skills')
        # print(desc)
    else:
        print(f'{i}: {offer_skills}')
        # offers_frame.loc[i, "RequiredSkills"] = ', '.join(offer_skills)

print(f'There are {with_no_skills} Offers with no skills available')
# offers_frame

1: {'c++', 'python', 'java', 'c'}
2: {'javascript', 'oracle', 'sql', 'angular', 'java'}
3: {'c++', 'python', 'c'}
4: {'html', 'javascript', 'angularjs', 'sql', 'c', 'css', '.net', 'java'}
5: {'.net', 'java'}
6: {'python', 'java'}
7: {'mysql', 'java'}
8: {'html', 'javascript', 'sql', 'jquery', 'c', 'css', '.net'}
9: {'docker', 'kubernetes', 'javascript', 'php', 'go', 'react', 'java'}
11: {'angular', 'react', 'aws', 'typescript'}
13: {'java'}
14: {'kubernetes', 'oracle', 'aws', 'react', 'angular', 'java'}
15: {'.net', 'c'}
16: {'mysql', 'sql', 'c', 'c++', 'python', 'java'}
17: {'javascript', 'flutter', 'oracle', 'vue.js', 'dart', 'aws', 'react', 'angular', 'typescript'}
18: {'c', '.net', 'sql'}
19: {'javascript', 'python', 'php', 'sql'}
20: {'c++', 'c'}
21: {'dart', 'flutter', 'aws'}
22: {'docker', 'kubernetes', 'sql', 'vite', 'c', '.net'}
23: {'php', 'java', 'c'}
24: {'python', 'aws', 'java', 'c'}
25: {'c++', 'java', 'c'}
26: {'c', '.net', 'mysql', 'sql'}
28: {'.net', 'sql'}
30: {'expre

### Tipo di Location
Per estrarre il tipo di Location dalle informazioni attualmente già presenti nel dataset, 
saranno interpretate come di tipo Remote tutte le offerte che non presentano una Location.

In [63]:
LOCATION_TYPES = {"Remote" : "Remote", "On Site" : "OnSite"}
locations_present = offers_frame["Location"].notna()
types = []

for is_present in locations_present:
    if is_present:
        types.append(LOCATION_TYPES["On Site"])
    else:
        types.append(LOCATION_TYPES["Remote"])

offers_frame.loc[:, "LocationType"] = types

offers_frame

,Name,Description,Location,LocationType
0,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria",OnSite
1,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona,OnSite
2,Software Developer,We are looking for talented and passionate peo...,55100 Lucca,OnSite
3,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata",OnSite
4,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia",OnSite
...,...,...,...,...
163,Scientist,Data: 9 gen 2024Dipartimento: Pre-ClinicalTe...,"Parma, Emilia-Romagna",OnSite
165,Data Scientist - Pharma,Akkodis è un leader globale nel mercato dell'i...,"Provincia di Torino, Piemonte",OnSite
166,DATA SCIENTIST | MACHINE LEARNING | BIG DATA,Techyon: Information Technology Recruitment Ex...,"Parma, Emilia-Romagna",OnSite
167,PDS AD&GMP Scientist I,Job DescriptionWhen you’re part of Thermo Fish...,20900 Monza,OnSite


### Linguaggi Richiesti
Per dedurre quali lingue parlate sono richieste da una data offerta di lavoro, viene utilizzato il modulo langdetect per identificare la lingua in cui è stata scritta la descrizione dell'offerta stessa, e successivamente tramite una ricerca all'interno del testo della descrizione per menzioni di ulteriori lingue.

In [64]:
from langdetect import detect
offers_langs = []
remove_map = {p:'' for p in punctuation}
supported_langs = ['italiano', 'inglese', 'italian', 'english']
eqs = [('italiano', 'italian'), ('inglese', 'english')]

for desc in offers_frame.Description:
    lang = detect(desc)
    if lang == 'it':
        lang = 'italiano'
    elif lang == 'en':
        lang = 'inglese'
    
    desc = remove_symbols(desc, remove_map)
    found = extract_symbols(desc, supported_langs)
    found.add(lang)
    found = list(found)
    
    for i, l in enumerate(found):
        for e in eqs:
            if l in e:
                found[i] = e[0]
                break
    
    offers_langs.append(found)
    
offers_frame.loc[:, "Languages"] = offers_langs

offers_frame

,Name,Description,Location,LocationType,Languages
0,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria",OnSite,"[inglese, inglese]"
1,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona,OnSite,[italiano]
2,Software Developer,We are looking for talented and passionate peo...,55100 Lucca,OnSite,"[inglese, inglese]"
3,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata",OnSite,[inglese]
4,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia",OnSite,[italiano]
...,...,...,...,...,...
163,Scientist,Data: 9 gen 2024Dipartimento: Pre-ClinicalTe...,"Parma, Emilia-Romagna",OnSite,[inglese]
165,Data Scientist - Pharma,Akkodis è un leader globale nel mercato dell'i...,"Provincia di Torino, Piemonte",OnSite,[italiano]
166,DATA SCIENTIST | MACHINE LEARNING | BIG DATA,Techyon: Information Technology Recruitment Ex...,"Parma, Emilia-Romagna",OnSite,[italiano]
167,PDS AD&GMP Scientist I,Job DescriptionWhen you’re part of Thermo Fish...,20900 Monza,OnSite,[inglese]
